In [1]:
import sys
import matplotlib
%matplotlib inline
import argparse
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
from notebooks.data_loader import *
from notebooks.helper_fxns import *
from notebooks.print_n_plot import *
from notebooks.build_network import *
from notebooks.train_val import *
import warnings
import lasagne
import theano
from theano import tensor as T
import sys
import numpy as np
import logging
import time
import pickle
import argparse
from os.path import join

importing Jupyter notebook from notebooks/data_loader.ipynb
importing Jupyter notebook from notebooks/helper_fxns.ipynb
importing Jupyter notebook from notebooks/print_n_plot.ipynb
importing Jupyter notebook from notebooks/build_network.ipynb
importing Jupyter notebook from notebooks/train_val.ipynb
importing Jupyter notebook from notebooks/objectives.ipynb


In [2]:
def setup_kwargs():
    
    default_args = {'input_shape': tuple([None] + [1, 50, 50]), 
                      'learning_rate': 0.01, 
                      'dropout_p': 0, 
                      'weight_decay': 0, #0.0001, 
                      'num_filters': 64, 
                      'num_fc_units': 32,
                      'num_layers': 4,
                      'momentum': 0.9,
                      'num_epochs': 10000,
                      'batch_size': 128,
                     "save_path": "None",
                    "num_events": 1000,
                    "sig_eff_at": 0.9996}
    
    
    # if inside a notebook, then get rid of weird notebook arguments, so that arg parsing still works
    if any(["jupyter" in arg for arg in sys.argv]):
        sys.argv=sys.argv[:1]


    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    #make a command line argument for every flag in default args
    for k,v in default_args.iteritems():
        parser.add_argument('--' + k, type=type(v), default=v, help=k)

    args = parser.parse_args()
    
    if args.save_path == "None":
        save_path = None
    else:
        save_path = args.save_path


    kwargs = default_args
    kwargs.update(args.__dict__)
    run_dir = create_run_dir(save_path)
    kwargs['save_path'] = run_dir
    kwargs["logger"] = get_logger(kwargs['save_path'])
    
    return kwargs

In [ ]:
if __name__ == "__main__":
    
    kwargs = setup_kwargs()
    
    h5_prefix = "/project/projectdirs/dasrepo/atlas_rpv_susy/hdf5/prod003_2016_11_14"
    
    dl = DataLoader(bg_cfg_file=[join(h5_prefix, "jetjet_JZ4.h5"),
                                 join(h5_prefix, "jetjet_JZ5.h5")],
                    sig_cfg_file=join(h5_prefix, "GG_RPV10_1400_850.h5"),
                    num_events=kwargs["num_events"], 
                    type_="hdf5",
                    use_premade=True)
    tr,val = dl.load_data()

    kwargs["logger"].info(str(kwargs))
    networks, fns = build_network(kwargs, build_layers(kwargs))
    tv = TrainVal(tr,val, kwargs, fns, networks)
    for epoch in range(kwargs["num_epochs"]):
        tv.do_one_epoch()
    
    
    

{'learning_rate': 0.01, 'num_layers': 4, 'input_shape': (None, 1, 50, 50), 'batch_size': 128, 'dropout_p': 0, 'num_events': 1000, 'num_epochs': 10000, 'weight_decay': 0, 'sig_eff_at': 0.9996, 'num_fc_units': 32, 'save_path': './results/run207', 'num_filters': 64, 'logger': <logging.Logger object at 0x7fe441f32a50>, 'momentum': 0.9}
<lasagne.layers.input.InputLayer object at 0x7fe42c410a10>(None, 1, 50, 50)
<lasagne.layers.conv.Conv2DLayer object at 0x7fe42c3d4b90>(None, 64, 50, 50)
<lasagne.layers.normalization.BatchNormLayer object at 0x7fe42c3c7f50>(None, 64, 50, 50)
<lasagne.layers.special.NonlinearityLayer object at 0x7fe42c3e9ad0>(None, 64, 50, 50)
<lasagne.layers.pool.MaxPool2DLayer object at 0x7fe42c3e9c10>(None, 64, 25, 25)
<lasagne.layers.conv.Conv2DLayer object at 0x7fe42c3e9c50>(None, 64, 25, 25)
<lasagne.layers.normalization.BatchNormLayer object at 0x7fe42c3e9e50>(None, 64, 25, 25)
<lasagne.layers.special.NonlinearityLayer object at 0x7fe42c3c5250>(None, 64, 25, 25)
<lasag

130722




<lasagne.layers.input.InputLayer object at 0x7fe42c410a10> : (None, 1, 50, 50)
<lasagne.layers.conv.Conv2DLayer object at 0x7fe42c3d4b90> : (None, 64, 50, 50)
<lasagne.layers.normalization.BatchNormLayer object at 0x7fe42c3c7f50> : (None, 64, 50, 50)
<lasagne.layers.special.NonlinearityLayer object at 0x7fe42c3e9ad0> : (None, 64, 50, 50)
<lasagne.layers.pool.MaxPool2DLayer object at 0x7fe42c3e9c10> : (None, 64, 25, 25)
<lasagne.layers.conv.Conv2DLayer object at 0x7fe42c3e9c50> : (None, 64, 25, 25)
<lasagne.layers.normalization.BatchNormLayer object at 0x7fe42c3e9e50> : (None, 64, 25, 25)
<lasagne.layers.special.NonlinearityLayer object at 0x7fe42c3c5250> : (None, 64, 25, 25)
<lasagne.layers.pool.MaxPool2DLayer object at 0x7fe42c3c5390> : (None, 64, 12, 12)
<lasagne.layers.conv.Conv2DLayer object at 0x7fe42c3c53d0> : (None, 64, 12, 12)
<lasagne.layers.normalization.BatchNormLayer object at 0x7fe42c3c55d0> : (None, 64, 12, 12)
<lasagne.layers.special.NonlinearityLayer object at 0x7fe42

beginning epoch 0


In [ ]:
# h5_prefix = "/project/projectdirs/dasrepo/atlas_rpv_susy/hdf5/prod003_2016_11_14"

# a=h5py.File(join(h5_prefix,"GG_RPV10_1400_850.h5" ))

# w=a["event_10"]["weight"]

# w.value

# g.value

# # x.shape

# # #test
# # x, y, xv,yv = load_train_val(num_events=100000)

# # def test_network(network_path):
# #     x_te, y_te = load_test()

# #     net = pickle.load(open(network_path))

# #     cfg = build_network(network_kwargs,net)
# #     return cfg['val_fn'](x_te, y_te)

# # network_path = './results/run84/model.pkl'



# # net = pickle.load(open(network_path))

# # cfg = build_network(network_kwargs,net)

# # y_pred = cfg['out_fn'](xv)

# # y_pred = y_pred[0]

# # best_sig = xv[np.argmax(y_pred[:,1])]

# # best_bg = xv[np.argmin(y_pred[:,1])]

# # plot_example(np.squeeze(best_sig))

# # plot_example(np.squeeze(best_bg))

# # inds = np.argsort(y_pred[:,1], axis=0)

# # best_bgs = np.squeeze(xv[inds[:25]])

# # best_sigs = np.squeeze(xv[inds[-26:-1]])

# # plot_examples(best_bgs,5, run_dir,"best_bg")

# # plot_examples(best_sigs,5, run_dir, "best_sig")

# # plot_filters(net,save_dir=run_dir)

# # plot_feature_maps(best_bgs[0], net, run_dir, name="best_bg")

# # best_bg = np.expand_dims(np.expand_dims(best_bgs[0], axis=0),axis=0)
# # best_sig = np.expand_dims(np.expand_dims(best_sigs[-1], axis=0),axis=0)
# # saliency_fn = compile_saliency_function(net)
# # saliency, max_class = saliency_fn(best_sig)
# # #np.squeeze(np.abs(saliency)).shape
# # show_images(best_sigs[-1], saliency, max_class, "default gradient", save_dir=run_dir)



